In [1]:
import sys
print(sys.executable)

C:\Users\risha\.conda\envs\pcm_pinn\python.exe


In [2]:
from fipy import Grid2D
print("FiPy ready inside notebook")


FiPy ready inside notebook


In [3]:
import os
import h5py
import numpy as np
from math import ceil
from datetime import datetime

from fipy import Grid2D, CellVariable, TransientTerm, DiffusionTerm, DefaultSolver


In [4]:
MATERIAL = {
    "rho": 780.0,
    "cp": 2000.0,
    "k": 0.2,
    "L": 2.0e5,
    "Tm": 50.0,
    "dT": 2.0
}


In [5]:
def liquid_fraction(T, Tm, dT):
    Ts, Tl = Tm - dT, Tm + dT
    f = np.zeros_like(T)
    f[T >= Tl] = 1.0
    mask = (T > Ts) & (T < Tl)
    xi = (T[mask] - Ts) / (2 * dT)
    f[mask] = 3 * xi**2 - 2 * xi**3
    return f

def df_dT(T, Tm, dT):
    Ts, Tl = Tm - dT, Tm + dT
    df = np.zeros_like(T)
    mask = (T > Ts) & (T < Tl)
    xi = (T[mask] - Ts) / (2 * dT)
    df[mask] = (6 * xi - 6 * xi**2) / (2 * dT)
    return df

In [6]:
# ============================================================
# 2D PCM SOLVER (OPTIMIZED)
# ============================================================

def run_2d_case(params, outpath, save_every=50):

    if os.path.exists(outpath):
        print(f"✔ Skipping existing file: {outpath}")
        return

    # ----------------------------
    # Geometry and mesh
    # ----------------------------
    Lx, Ly = params["Lx"], params["Ly"]
    nx, ny = params["nx"], params["ny"]
    dx, dy = Lx / nx, Ly / ny

    mesh = Grid2D(nx=nx, ny=ny, dx=dx, dy=dy)

    # Cell center coordinates
    x = np.linspace(dx/2, Lx - dx/2, nx)
    y = np.linspace(dy/2, Ly - dy/2, ny)

    # ----------------------------
    # Variables
    # ----------------------------
    T = CellVariable(mesh=mesh, value=params["T_init"])
    c_eff = CellVariable(mesh=mesh, value=params["cp"])
    q_src = CellVariable(mesh=mesh, value=0.0)

    eq = (
        TransientTerm(coeff=params["rho"] * c_eff)
        == DiffusionTerm(coeff=params["k"]) + q_src
    )

    # ----------------------------
    # Time loop
    # ----------------------------
    dt = params["dt"]
    nsteps = int(ceil(params["t_total"] / dt))

    times, T_hist, f_hist = [], [], []

    Xc, Yc = np.meshgrid(x, y)
    x_flat = Xc.ravel()

    for step in range(nsteps + 1):
        t = step * dt

        # Apparent heat capacity
        c_eff.value = (
            params["cp"]
            + params["L"] * df_dT(T.value, params["Tm"], params["dT"])
        )

        # Heat flux
        hf = params["heat_flux"](t) if callable(params["heat_flux"]) else params["heat_flux"]

        q = np.zeros(mesh.numberOfCells)
        left_cells = x_flat <= dx * 0.6
        q[left_cells] = hf / dx
        q_src.value = q

        # Solve
        eq.solve(var=T, dt=dt, solver=DefaultSolver())

        # Save snapshots
        if step % save_every == 0 or step == nsteps:
            times.append(t)
            T2D = T.value.reshape((ny, nx))
            T_hist.append(T2D)
            f_hist.append(liquid_fraction(T2D, params["Tm"], params["dT"]))

    # ----------------------------
    # Save to HDF5
    # ----------------------------
    with h5py.File(outpath, "w") as hf:
        g = hf.create_group("case")
        g.create_dataset("x", data=x)
        g.create_dataset("y", data=y)
        g.create_dataset("times", data=np.array(times))
        g.create_dataset("T", data=np.array(T_hist))
        g.create_dataset("f", data=np.array(f_hist))

        meta = g.create_group("params")
        for k, v in params.items():
            meta.attrs[k] = v if not callable(v) else "callable"
        meta.attrs["created"] = datetime.now().isoformat()

    print(f"✔ Saved {outpath}")

In [7]:
# ============================================================
# PARAMETER SWEEP (2D DATASET)
# ============================================================

def generate_dataset_2d(output_dir="pcm_dataset_2D", ncases=100):

    os.makedirs(output_dir, exist_ok=True)

    heat_fluxes = np.linspace(500, 6000, 8)
    lengths = np.linspace(0.01, 0.04, 6)
    T_inits = [20.0, 30.0, 40.0]

    case_id = 0

    for q in heat_fluxes:
        for L in lengths:
            for T0 in T_inits:

                if case_id >= ncases:
                    return

                def heat_schedule(t, q=q):
                    return q if (t % 2400) < 1200 else 0.0

                params = {
                    "Lx": float(L),
                    "Ly": float(L / 2),
                    "nx": 80,
                    "ny": 40,
                    "t_total": 3600.0,
                    "dt": 1.0,
                    "T_init": float(T0),
                    "heat_flux": heat_schedule,
                    **MATERIAL
                }

                fname = f"pcm2D_case_{case_id:04d}.h5"
                outpath = os.path.join(output_dir, fname)

                run_2d_case(params, outpath)
                case_id += 1


In [8]:
print("🚀 Starting 2D PCM dataset generation...")
generate_dataset_2d(ncases=100)
print("✅ 2D PCM dataset generation complete.")


🚀 Starting 2D PCM dataset generation...
✔ Saved pcm_dataset_2D\pcm2D_case_0000.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0001.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0002.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0003.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0004.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0005.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0006.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0007.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0008.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0009.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0010.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0011.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0012.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0013.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0014.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0015.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0016.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0017.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0018.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0019.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0020.h5
✔ Saved pcm_dataset_2D\pcm2D_case_0021.h5
✔ Saved pcm_dataset_2D\pcm2D_case_00